In [15]:
!pip install vncorenlp


[notice] A new release of pip available: 22.3 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
from vncorenlp import VnCoreNLP

rdrsegmenter = VnCoreNLP(
    r"D:\Download\VnCoreNLP-1.1.1\VnCoreNLP-1.1.1\VnCoreNLP-1.1.1.jar",
    annotators="wseg",
    max_heap_size='-Xmx2g'
)
print(rdrsegmenter.tokenize("Hồ Chí Minh là lãnh tụ vĩ đại"))

[['Hồ_Chí_Minh', 'là', 'lãnh_tụ', 'vĩ_đại']]


In [17]:
input_file = r"D:\NCKH\estimate_question_difficulty\Tokenizer\qa_processed.txt"
output_file = r"D:\NCKH\estimate_question_difficulty\Tokenizer\qa_processed_ws.txt"

with open(input_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

with open(output_file, "w", encoding="utf-8") as f:
    for line in lines:
        line = line.strip()
        if line:
            segmented = rdrsegmenter.tokenize(line)
            f.write(" ".join(segmented[0]) + "\n")

In [18]:
import json
import re

# Đọc file txt
with open(r"D:\NCKH\estimate_question_difficulty\Tokenizer\qa_processed_ws.txt", "r", encoding="utf-8") as f:
     text = f.read()

# Tách các block câu hỏi bằng chuỗi gạch ngang dài
blocks = [b.strip() for b in text.split("- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -") if b.strip()]


questions = []
for block in blocks:
    # Lấy môn
    subject_match = re.search(r"Môn\s*:\s*(\S+)", block)
    subject = subject_match.group(1).strip() if subject_match else None

    # Lấy câu hỏi
    question_match = re.search(r"Câu hỏi\s*:\s*(.+)", block)
    question = question_match.group(1).strip() if question_match else None

    # Lấy đáp án
    options = re.findall(r"\d+\s*\.\s*([A-D])\s+(.+)", block)
    options_dict = {opt[0]: opt[1].strip() for opt in options} if options else None

    # Lấy đáp án đúng
    answer_match = re.search(r"Đáp_án đúng\s*:\s*([A-D])", block)
    answer = answer_match.group(1).strip() if answer_match else None

    if subject and question and options_dict and answer:
        questions.append({
            "subject": subject,
            "question": question,
            "options": options_dict,
            "answer": answer
        })

# Xuất ra JSON
with open("questions.json", "w", encoding="utf-8") as f:
    json.dump(questions, f, ensure_ascii=False, indent=4)

print(f"✅ Đã chuyển thành công {len(questions)} câu hỏi sang questions.json")

✅ Đã chuyển thành công 149 câu hỏi sang questions.json
